Shar-Detector Notebook

In [ ]:
#%%writefile Shark-Detector
## This is the current iteration of this program ignore others 

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import data as tf_data
import keras
from keras import layers 

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

dataset_path = 'C:/Users/brodi/Desktop/CV-Project'
shark_path = os.path.join(dataset_path, 'shark')
no_shark_path = os.path.join(dataset_path, 'no_shark')
validation_path = os.path.join(dataset_path, 'Test-Bench-1')

if not os.path.exists(shark_path):
    raise FileNotFoundError(f"Shark Path not found")
if not os.path.exists(no_shark_path):
    raise FileNotFoundError(f"No_Shark Path not found")
    

def preprocess(filepath):
    img = cv2.imread(filepath)
    if img is None:
        print(f"Failed to load image: {filepath}")
        return None
    img = cv2.resize(img, (64, 64))
    img = img/255.0
    return img


def load(shark_path, no_shark_path):
    shark_images = [preprocess(os.path.join(shark_path, img)) for img in os.listdir(shark_path)
                    if img.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp', '.gif', '.tif', '.tiff', '.jfif', '.avif'))]
    no_shark_images = [preprocess(os.path.join(no_shark_path, img)) for img in os.listdir(no_shark_path)
                       if img.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp', '.gif', '.tif', '.tiff', '.jfif', '.avif'))]
    shark_images = [img for img in shark_images if img is not None]
    no_shark_images = [img for img in no_shark_images if img is not None]
    shark_labels = [1] * len(shark_images)
    no_shark_labels = [0] * len(no_shark_images)
    images = np.array(shark_images + no_shark_images)
    labels = np.array(shark_labels + no_shark_labels)
    return images, labels

images, labels = load(shark_path, no_shark_path)
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)
print(f"Number of training images: {len(train_images)}")
print(f"Number of validation images: {len(val_images)}")


base_model = tf.keras.applications.EfficientNetB0(
    input_shape=(64, 64, 3),
    include_top=False,
    weights=None
)

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.BatchNormalization(), 
    tf.keras.layers.Dense(256),
    tf.keras.layers.LeakyReLU(alpha=0.1), 
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_images,
    train_labels,
    validation_data=(val_images, val_labels),
    epochs=24,
    batch_size=8
)
print("Model trained successfully!")
print("\nSaving model...")
model.save('shark_detector.keras')
print("Model saved successfully!")







In [ ]:
#%writefile Data_Generator
from better_bing_image_downloader import downloader
query_string = 'aerial view of the ocean -shark -cityscapes'

downloader(query_string, limit=100, output_dir='dataset', adult_filter_off=True,
force_replace=False, timeout=60, filter="", verbose=True, badsites= [], name='DataSet1')